In [ ]:
import numpy as np
from env_1 import Env1
#from algos import *
import math
from scipy.stats import norm

#np.random.seed(42)

In [ ]:
def generate_proposal(prev_theta, step_size):

    # Set the parameters for the 2D normal distribution


    covariance = 0  # covariance between x and y


    cov_matrix = [[step_size**2, covariance], [covariance, step_size**2]]

    samples = np.random.multivariate_normal(prev_theta, cov_matrix, 1)[0]

    return samples

In [ ]:
samples = []

for i in range(50):

    theta1 = generate_proposal([1, 0.5], step_size=0.01)
    samples.append(theta1)

In [ ]:
import numpy as np
import random

#np.random.seed(42)

def rand_demo(xi):
    n, m = np.shape(xi)
    xi1 = np.copy(xi)
    for idx in range(1, n-1):

        # range of workspace in each axis
        x = np.random.uniform(0.3, 0.75)
        y = np.random.uniform(-0.1, 0.5)
        z = np.random.uniform(0.1, 0.6)
        #x = np.random.uniform(0.4, 0.65)
        #y = np.random.uniform(0, 0.3)
        #z = np.random.uniform(0, 0.3)
        xi1[idx, :] = np.array([x, y, z])
    return xi1


In [ ]:
def human_demo_2(env, xi, theta, n_samples, n_demos=10, filter=True):
    XI = []
    Fs = []
    Rs = []
    
    best_fs = None
    best_xi = None
    max_reward = np.NINF
    for idx in range(n_samples):
        xi1 = rand_demo(xi)
        
        f = env.feature_count(xi1)

        if (np.abs(f[0])/np.abs(f[1]) > 50) or (np.abs(f[1])/np.abs(f[0]) > 50):
            print("Bad demo: ", f)
            continue
        
        else:
            print("Good demo: ", f)
            R = env.reward(f, theta)

            if R > max_reward:
                max_reward = R
                best_fs = f
                best_xi = xi1

            Rs.append(R)
            Fs.append(f)
            XI.append(xi1)

    # Convert lists to numpy arrays
    Rs = np.array(Rs)
    XI = np.array(XI)
    Fs = np.array(Fs)

    # Get indices that would sort Rs in descending order
    sorted_indices = np.argsort(Rs)[::-1]

    # Sort XI, Fs, and Rs based on these indices
    XI = XI[sorted_indices]
    Fs = Fs[sorted_indices]
    Rs = Rs[sorted_indices]

    # Return the top n_demos (default is 10) elements with the highest R
    top_XI = XI[:n_demos]
    top_Fs = Fs[:n_demos]
    top_Rs = Rs[:n_demos]

    return top_XI, best_xi, top_Fs, best_fs

In [ ]:
num_generated_demos = 100

num_demonstrations = 10
beta = 0.1

start_position = np.array([0.4, 0.0, 0.25])
goal_position = np.array([0.75, 0.0, 0.1])
xi0 = np.linspace(start_position, goal_position, 3)

true_reward_param = np.array([1, 0.1, 0.01])

# initialize environment
env = Env1(visualize=True)

demonstrations, xi_star, trajectory_space, f_star = human_demo_2(env, xi0, true_reward_param , n_samples=num_generated_demos, n_demos=num_demonstrations)## this the problem, I should fix the trajectories before the loop
#env.play_traj(xi_star, T=2.0)